In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

%matplotlib inline

---
**Update**: Cleaning the file before pushing to SQL for continuos project

---

In [49]:
df = pd.read_excel("Document Recieved\dec_raw_transaction_iNet.xlsx", engine="openpyxl",sheet_name="Arbeitsblatt exportieren")

In [50]:
df.drop(columns=['Fall','Bill receiver id','Bill receiver name','Cost text','Currency','Vat code','Vat rate'],inplace=True)

In [51]:
df["Cost rate"] = df["Cost rate"].astype(str)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341430 entries, 0 to 341429
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Bill no                341430 non-null  object 
 1   Bill date              341430 non-null  object 
 2   Bill sender id         341430 non-null  object 
 3   Bill sender name       341430 non-null  object 
 4   Load no                341430 non-null  int64  
 5   TO no                  341430 non-null  object 
 6   Country TO consignor   341430 non-null  object 
 7   Zip TO consignor       341430 non-null  object 
 8   City TO consignor      341430 non-null  object 
 9   Name TO consignor      341430 non-null  object 
 10  Country TO consignee   341430 non-null  object 
 11  Zip TO consignee       341430 non-null  object 
 12  City TO consignee      341430 non-null  object 
 13  Name TO consignee      341430 non-null  object 
 14  Country leg consignor  341430 non-nu

In [54]:
# type of cost calculation
df["uom_type"] = df.apply(lambda row: "lm" if "lm" in row["Cost rate"].lower() \
                              else ('kg' if 'kg' in row["Cost rate"].lower() and row['Cost desc']!= 'Handling Cost (X-dock)' 
                                        else 'not_lm_kg'), axis=1)

In [55]:
df[df['Cost desc']=='Handling Cost (X-dock)']

,Bill no,Bill date,Bill sender id,Bill sender name,Load no,TO no,Country TO consignor,Zip TO consignor,City TO consignor,Name TO consignor,Country TO consignee,Zip TO consignee,City TO consignee,Name TO consignee,Country leg consignor,Zip leg consignor,City leg consignor,Name leg consignor,Country leg consignee,Zip leg consignee,City leg consignee,Name leg consignee,Processing date,Pickup date,Weight TO [kg],Volume TO [m3],Loading meter leg,Transport mode leg,Cost type,Cost desc,Cost rate,Amount,uom_type
297343,27021,01.12.2021,SCAC12-6,DHL SE X-dock,1688567,22984584,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,SE,572 36,OSKARSHAMN,Scania CV AB,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,SE,556 52,JONKOPING,DHL (SE) X-DOCK JONKOPING,23.11.2021,16.11.2021,92.0,0.605,0.134,LTL,SCAHCO,Handling Cost (X-dock),84538.0 HU,2.06,not_lm_kg
297344,27021,01.12.2021,SCAC12-6,DHL SE X-dock,1688569,22984966,NO,3647,HVITTINGFOSS,KONGSBERG AUTOMOTIVE AS,SE,151 38,SODERTALJE,Scania CV AB,NO,3647,HVITTINGFOSS,KONGSBERG AUTOMOTIVE AS,SE,556 52,JONKOPING,DHL (SE) X-DOCK JONKOPING,18.11.2021,16.11.2021,1166.0,3.799,0.616,LTL,SCAHCO,Handling Cost (X-dock),84538.0 HU,10.80,not_lm_kg
297345,27021,01.12.2021,SCAC12-6,DHL SE X-dock,1682731,10160173,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,PL,76-200,SLUPSK,Scania CV AB,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,SE,556 52,JONKOPING,DHL (SE) X-DOCK JONKOPING,16.11.2021,09.11.2021,25.0,0.170,0.040,LTL,SCAHCO,Handling Cost (X-dock),84538.0 HU,0.61,not_lm_kg
297346,27021,01.12.2021,SCAC12-6,DHL SE X-dock,1676907,22961177,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,BE,3660,OUDSBERGEN,Scania CV AB,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,SE,556 52,JONKOPING,DHL (SE) X-DOCK JONKOPING,09.11.2021,02.11.2021,138.0,0.516,0.127,LTL,SCAHCO,Handling Cost (X-dock),84538.0 HU,1.95,not_lm_kg
297347,27021,01.12.2021,SCAC12-6,DHL SE X-dock,1691915,22991374,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,BE,3660,OUDSBERGEN,Scania CV AB,NO,2830,RAUFOSS,KONGSBERG AUTOMOTIVE AS,SE,556 52,JONKOPING,DHL (SE) X-DOCK JONKOPING,26.11.2021,19.11.2021,249.0,1.723,0.293,LTL,SCAHCO,Handling Cost (X-dock),84538.0 HU,4.90,not_lm_kg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341425,27453,31.12.2021,SCAC26-2,BREGER X-dock Luisant,1713892,23033601,ES,46136,"MUSEROS, VALENCIA, COMUNIDAD VALENCIAN",KAMAX S.L.U,NL,8041 AL,ZWOLLE,SCANIA CV AB,ES,20305,IRUN,Schenker ES (ES) X-DOCK IRUN,FR,77564,LIEUSAINT,BREGER (FR) X-DOCK LIEUSAINT,17.12.2021,16.12.2021,2138.0,2.504,0.417,LTL,SCAHCO,Handling Cost (X-dock),2138.0 kg. (9.5 per 1000.0),20.31,not_lm_kg
341426,27453,31.12.2021,SCAC26-2,BREGER X-dock Luisant,1713892,23034929,ES,31012,PAMPLONA,TRW AUTOMOTIVE ESPA??A SL,NL,8041 AL,Zwolle,Scania Production Zwolle,ES,20305,IRUN,Schenker ES (ES) X-DOCK IRUN,FR,77564,LIEUSAINT,BREGER (FR) X-DOCK LIEUSAINT,17.12.2021,16.12.2021,2320.0,4.482,0.715,LTL,SCAHCO,Handling Cost (X-dock),2320.0 kg. (9.5 per 1000.0),22.04,not_lm_kg
341427,27453,31.12.2021,SCAC26-2,BREGER X-dock Luisant,1713892,23032846,ES,48260,ERMUA,ESTAMPACIONES METALICAS EGUI S.A.U.,BE,3660,OUDSBERGEN,Scania CV AB,ES,20305,IRUN,Schenker ES (ES) X-DOCK IRUN,FR,77564,LIEUSAINT,BREGER (FR) X-DOCK LIEUSAINT,17.12.2021,16.12.2021,1979.0,7.468,1.389,LTL,SCAHCO,Handling Cost (X-dock),2569.65 kg. (9.5 per 1000.0),24.41,not_lm_kg
341428,27453,31.12.2021,SCAC26-2,BREGER X-dock Luisant,1713892,23031486,ES,12540,VILLARREAL (CASTELLON),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",FR,49102,Angers,SCANIA CV AB,ES,20305,IRUN,Schenker ES (ES) X-DOCK IRUN,FR,77564,LIEUSAINT,BREGER (FR) X-DOCK LIEUSAINT,17.12.2021,16.12.2021,997.0,2.816,0.807,LTL,SCAHCO,Handling Cost (X-dock),1492.95 kg. (9.5 per 1000.0),14.18,not_lm_kg


In [56]:
def add_uom_v(row):
    if row['uom_type'] == 'kg':
        word_index = row['Cost rate'].lower().index('kg')
        return row['Cost rate'][:word_index].strip()
    elif row['uom_type'] == 'lm':
        word_index = row['Cost rate'].lower().index('lm')
        return row['Cost rate'][:word_index].strip()
    else:
        return 0
    
    
    
# type of cost calculation
df["uom_value"] = df.apply(add_uom_v, axis=1)

In [57]:
df.head()

,Bill no,Bill date,Bill sender id,Bill sender name,Load no,TO no,Country TO consignor,Zip TO consignor,City TO consignor,Name TO consignor,Country TO consignee,Zip TO consignee,City TO consignee,Name TO consignee,Country leg consignor,Zip leg consignor,City leg consignor,Name leg consignor,Country leg consignee,Zip leg consignee,City leg consignee,Name leg consignee,Processing date,Pickup date,Weight TO [kg],Volume TO [m3],Loading meter leg,Transport mode leg,Cost type,Cost desc,Cost rate,Amount,uom_type,uom_value
0,27216,10.12.2021,SCAC81,Jan De Rijk,1708024,23020651,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,NL,2231,Rijnsburg,Alutech Rijnsburg,NL,8041 AL,Zwolle,Scania Production Zwolle,08.12.2021,08.12.2021,2560.0,78.720,13.600,FTL,SCAPAD,Only Inbound Invoicing Users - Price Adjustment,nan,1206.85,not_lm_kg,0
1,27034,03.12.2021,SCAC10-13,DSV ES,1695101,30239604,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,12540,VILLARREAL (CASTELLÓN),"MUELLES Y BALLESTAS HISPANO-ALEMANAS, S.L.U.",29.11.2021,24.11.2021,12845.0,69.750,11.800,LTL,SCAMAT,Maut,23226.8835 kg.,92.90,kg,23226.8835
2,27034,03.12.2021,SCAC10-13,DSV ES,1695100,30239615,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,29.11.2021,24.11.2021,13812.0,72.853,13.262,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0
3,27034,03.12.2021,SCAC10-13,DSV ES,1700008,30240209,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,19004,GUADALAJARA,"NEOVIA LOGISTICS SERVICES SPAIN, S.A.",DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,03.12.2021,30.11.2021,14772.0,71.332,13.600,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0
4,27034,03.12.2021,SCAC10-13,DSV ES,1699921,30240157,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,DE,52249,ESCHWEILER,LEADEC HOLDING GMBH,ES,48260,"ERMUA, POLIGONO INDUSTRIAL URTIA",ESTAMPACIONES METALICAS EGUI S.A.U.,03.12.2021,30.11.2021,8654.0,75.361,13.304,LTL,SCAMAT,Maut,24499.0 kg.,92.90,kg,24499.0


In [3]:
#df.to_excel('dec_iNet_cleaned.xlsx',index=False)